In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


# Get train, test, and val paths of images
This code relies on the notebook being started from a bash shell

In [3]:
train_viral = !find 'data/chest_xray/train/PNEUMONIA/' -regex '.*virus.*'
train_bacterial = !find 'data/chest_xray/train/PNEUMONIA/' -regex '.*bacteria.*'
train_normal = !ls 'data/chest_xray/train/NORMAL'
train_normal = ['data/chest_xray/train/NORMAL/' + file for file in train_normal]

test_viral = !find 'data/chest_xray/test/PNEUMONIA/' -regex '.*virus.*'
test_bacterial = !find 'data/chest_xray/test/PNEUMONIA/' -regex '.*bacteria.*'
test_normal = !ls 'data/chest_xray/test/NORMAL'
test_normal = ['data/chest_xray/test/NORMAL/' + file for file in test_normal]

val_viral = !find 'data/chest_xray/val/PNEUMONIA/' -regex '.*virus.*'
val_bacterial = !find 'data/chest_xray/val/PNEUMONIA/' -regex '.*bacteria.*'
val_normal = !ls 'data/chest_xray/val/NORMAL'
val_normal = ['data/chest_xray/val/NORMAL/' + file for file in val_normal]

Code from Grace's EDA notebook

In [4]:
# Creating an datagenerator object that will perform image augmentation
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=90,
                                   width_shift_range=.2,
                                   height_shift_range=.2,
                                   shear_range=.2,
                                   zoom_range=.2,
                                   horizontal_flip=True,
                                   brightness_range=[.5, 1.5])

# Datagenerators for the test and validation set will only rescale the images
test_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)


# Applying the datagenerator objects to the images in the folders
image_size = (96, 96)
training_set = train_datagen.flow_from_directory('data/chest_xray/train',
                                                 target_size=image_size,
                                                 batch_size=32,
                                                 class_mode='binary',
                                                 color_mode='grayscale')

val_set = val_datagen.flow_from_directory('data/chest_xray/val',
                                          target_size=image_size,
                                          batch_size=32,
                                          class_mode='binary',
                                          color_mode='grayscale')

test_set = test_datagen.flow_from_directory('data/chest_xray/test',
                                            target_size=image_size,
                                            batch_size=32,
                                            class_mode='binary', 
                                            color_mode='grayscale')

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [5]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(96, 96, 1)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(32, (4, 4), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['acc'])

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 94, 94, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 47, 47, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 44, 44, 32)        16416     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 22, 22, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 20, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6400)              0

In [7]:
tf.debugging.set_log_device_placement(True)

with tf.device('/CPU:0'):
    results = model.fit(training_set, batch_size=32, validation_data=val_set, epochs=15)

Epoch 1/15
163/163 [==============================] - 147s 898ms/step - loss: 0.5370 - acc: 0.7362 - val_loss: 0.6680 - val_acc: 0.4375
Epoch 2/15
163/163 [==============================] - 119s 726ms/step - loss: 0.4795 - acc: 0.7402 - val_loss: 0.7004 - val_acc: 0.5000
Epoch 3/15
163/163 [==============================] - 92s 563ms/step - loss: 0.4626 - acc: 0.7427 - val_loss: 0.6437 - val_acc: 0.5000
Epoch 4/15
163/163 [==============================] - 92s 565ms/step - loss: 0.4573 - acc: 0.7475 - val_loss: 1.0161 - val_acc: 0.5000
Epoch 5/15
163/163 [==============================] - 93s 571ms/step - loss: 0.4286 - acc: 0.7812 - val_loss: 0.6127 - val_acc: 0.6250
Epoch 6/15
163/163 [==============================] - 94s 573ms/step - loss: 0.3586 - acc: 0.8307 - val_loss: 0.6047 - val_acc: 0.6875
Epoch 7/15
163/163 [==============================] - 92s 564ms/step - loss: 0.3402 - acc: 0.8485 - val_loss: 0.8408 - val_acc: 0.7500
Epoch 8/15
163/163 [==============================] -

In [8]:
second_model = models.Sequential()
second_model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(96, 96, 1)))
second_model.add(layers.MaxPooling2D((2, 2)))

second_model.add(layers.Conv2D(32, (4, 4), activation='relu'))
second_model.add(layers.MaxPooling2D((2, 2)))

second_model.add(layers.Conv2D(64, (3, 3), activation='relu'))
second_model.add(layers.MaxPooling2D((2, 2)))

second_model.add(layers.Flatten())
second_model.add(layers.Dense(64, activation='relu'))
second_model.add(layers.Dense(64, activation='relu'))
second_model.add(layers.Dense(1, activation='sigmoid'))

second_model.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['acc'])

In [9]:
print(tf.config.list_physical_devices())


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [10]:
with tf.device('/GPU:0'):
    results = second_model.fit(training_set, batch_size=32, validation_data=val_set, epochs=15)

Epoch 1/15
163/163 [==============================] - 180s 986ms/step - loss: 0.5447 - acc: 0.7429 - val_loss: 1.1671 - val_acc: 0.5000
Epoch 2/15
163/163 [==============================] - 106s 650ms/step - loss: 0.4910 - acc: 0.7345 - val_loss: 0.7562 - val_acc: 0.5000
Epoch 3/15
163/163 [==============================] - 93s 571ms/step - loss: 0.4638 - acc: 0.7412 - val_loss: 0.9281 - val_acc: 0.4375
Epoch 4/15
163/163 [==============================] - 89s 547ms/step - loss: 0.4546 - acc: 0.7502 - val_loss: 0.7560 - val_acc: 0.5625
Epoch 5/15
163/163 [==============================] - 87s 536ms/step - loss: 0.4626 - acc: 0.7477 - val_loss: 0.7386 - val_acc: 0.4375
Epoch 6/15
163/163 [==============================] - 78s 478ms/step - loss: 0.4414 - acc: 0.7678 - val_loss: 0.7477 - val_acc: 0.5000
Epoch 7/15
163/163 [==============================] - 77s 470ms/step - loss: 0.4171 - acc: 0.7937 - val_loss: 0.6697 - val_acc: 0.7500
Epoch 8/15
163/163 [==============================] -